In [1]:
# Import the Pandas library

import openpyxl
import io
import tempfile
import pandas as pd
import numpy as np
import sys

sys.path.append('../')

from models.linear_regression import LinReg

lin_reg = LinReg()
path = "../.test_data/more_data.xlsx"

coded_df = lin_reg.process_excel(path)


print(coded_df.info())
coded_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42432 entries, 0 to 42431
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NAME             42432 non-null  object 
 1   date             42432 non-null  int64  
 2   color_and_value  42432 non-null  object 
 3   name_code        42432 non-null  int64  
 4   value            42432 non-null  float64
 5   color            42432 non-null  object 
 6   color_code       42432 non-null  int64  
 7   next_color_code  42296 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 2.6+ MB
None


,NAME,date,color_and_value,name_code,value,color,color_code,next_color_code
0,GJ1,1,28.26107 | A9D08E,1,28.261070,A9D08E,0,0.0
1,GJ2,1,47.95346 | A9D08E,2,47.953460,A9D08E,0,0.0
2,GJ3,1,78.28529 | 9BC2E6,3,78.285290,9BC2E6,0,0.0
3,GJ4,1,47.75716 | A9D08E,4,47.757160,A9D08E,0,0.0
4,GJ5,1,16.25788 | A9D08E,5,16.257880,A9D08E,0,0.0
...,...,...,...,...,...,...,...,...
42427,GJ132,312,13.32481 | D5A6BD,132,13.324810,D5A6BD,1,NaN
42428,GJ133,312,-3.43532 | A9D08E,133,-3.435320,A9D08E,0,NaN
42429,GJ134,312,-1.26767 | A9D08E,134,-1.267670,A9D08E,0,NaN
42430,GJ135,312,5.563966 | 8EA9DB,135,5.563966,8EA9DB,0,NaN


In [2]:
coded_df[coded_df['color_code'] == 1].shape


(10624, 8)

In [3]:
lin_reg.last_df

,NAME,2024-04-01 00:00:00
0,GJ1,-1.41278 | FFFF00
1,GJ2,-15.1445 | FFFF00
2,GJ3,-10.8412 | A9D08E
3,GJ4,-0.56551 | 8EA9DB
4,GJ5,44.2933 | FFC000
...,...,...
131,GJ132,13.32481 | D5A6BD
132,GJ133,-3.43532 | A9D08E
133,GJ134,-1.26767 | A9D08E
134,GJ135,5.563966 | 8EA9DB


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split



# Splitting the dataset into the Training set and Test set
X , y = lin_reg.train_test_split(coded_df)


X.shape, y.shape

((42296, 4), (42296,))

In [5]:
# Create and train the model


lin_reg.fit(X, y)

print(f'mse : {lin_reg.mse}' )

print(f'accuracy : {lin_reg.accuracy}' )


mse : 0.25472813238770686
accuracy : 0.7452718676122931


In [6]:
# make predictions
df = lin_reg.predict_last()

df

,NAME,2024-04-01 00:00:00,color_and_value,next_color_code
0,GJ1,-1.41278 | FFFF00,-1.41278 | FFFF00,0
1,GJ2,-15.1445 | FFFF00,-15.1445 | FFFF00,0
2,GJ3,-10.8412 | A9D08E,-10.8412 | A9D08E,0
3,GJ4,-0.56551 | 8EA9DB,-0.56551 | 8EA9DB,0
4,GJ5,44.2933 | FFC000,44.2933 | FFC000,0
...,...,...,...,...
131,GJ132,13.32481 | D5A6BD,13.32481 | D5A6BD,0
132,GJ133,-3.43532 | A9D08E,-3.43532 | A9D08E,0
133,GJ134,-1.26767 | A9D08E,-1.26767 | A9D08E,0
134,GJ135,5.563966 | 8EA9DB,5.563966 | 8EA9DB,0


In [7]:
df[(df['next_color_code'] == 1) ].shape

(0, 4)